In [4]:
# !unzip FMD.zip

In [5]:

from sklearn.preprocessing import normalize

import os
import itertools

import numpy as np
import pandas as pd
import seaborn as sn
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from skimage.feature import greycomatrix , greycoprops
from skimage.io import imread
from skimage import img_as_ubyte





In [6]:
def is_jpg(file):
    extention = file.split(sep=".")[-1]
    return file if extention == "jpg" else None

In [7]:
def filter_jpgs(files):

    filtered_files = [file for file in files if is_jpg(file)]
    return filtered_files

In [8]:
def images_paths(dataset_folder):

    existing_folders = sorted(os.listdir(f'{dataset_folder}'))
    print(f'Existing Folders Or Labels are: {", ".join(existing_folders)}')
    
    paths_dict = {folder : sorted(os.listdir(f'{path}/{folder}')) for folder in existing_folders } # { folder : its_images }
    print(f'First Folder is: { list(paths_dict.keys())[0] }, Its First Two Images: { list(paths_dict.values())[0][:2] } :')
    
    images_paths = np.array([], dtype = str)
    
    for folder , files in paths_dict.items():
        cur_folder_paths = [f'{dataset_folder}/{folder}/{file}' for file in files if is_jpg(file) ]
        images_paths = np.concatenate( (images_paths, cur_folder_paths), axis=0 )
    
    return images_paths

In [9]:
def images_labels(dataset_folder):
    data_folders = sorted(os.listdir(f'{dataset_folder}'))
    
    labels = np.array([], dtype = np.uint8)

    for i,folder in enumerate(data_folders):
        images = filter_jpgs( sorted(os.listdir(f'{path}/{folder}') ) )
        cur_labels = np.full( len(images) , i)
        labels = np.concatenate( (labels, cur_labels)  , axis = 0 )
        
    return labels

In [10]:
path = 'image'

images_paths = images_paths(path)
images_labels = images_labels(path)

Existing Folders Or Labels are: fabric, foliage, glass, leather, metal, paper, plastic, stone, water, wood
First Folder is: fabric, Its First Two Images: ['fabric_moderate_001_new.jpg', 'fabric_moderate_002_new.jpg'] :


In [11]:
images_paths[:10]

array(['image/fabric/fabric_moderate_001_new.jpg',
       'image/fabric/fabric_moderate_002_new.jpg',
       'image/fabric/fabric_moderate_003_new.jpg',
       'image/fabric/fabric_moderate_004_new.jpg',
       'image/fabric/fabric_moderate_005_new.jpg',
       'image/fabric/fabric_moderate_006_new.jpg',
       'image/fabric/fabric_moderate_007_new.jpg',
       'image/fabric/fabric_moderate_008_new.jpg',
       'image/fabric/fabric_moderate_009_new.jpg',
       'image/fabric/fabric_moderate_010_new.jpg'], dtype='<U42')

In [12]:
images_labels[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
import cv2





ar=[]
for path in images_paths:
  # Convert to Grayscale
  img_gray=cv2.imread(path,0)
  print(img_gray.shape)
  ar.append(img_gray)

(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)
(384, 512)

In [14]:
ar[0].shape

(384, 512)

In [15]:
gray = np.array(ar)

X = np.array(ar)
Y = np.array(images_labels)
print(X.shape)

(1000, 384, 512)


In [16]:
gray.shape


(1000, 384, 512)

In [18]:
def GLCM(d,angle,images):
    features = []
    for image in images:
        glcm = greycomatrix(image, distances=d, angles=angle, levels=256)
        contrast = greycoprops(glcm, 'contrast')[0, 0]
        energy = greycoprops(glcm, 'energy')[0, 0]
        homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
        correlation = greycoprops(glcm, 'correlation')[0, 0]
        features.append([contrast, energy, homogeneity, correlation])
    return features

In [19]:
g= np.zeros((256,256))
im = X[0]
for i in range(im.shape[0]):
    for j in range(im.shape[1]-1):
        g[im[i][j]][im[i][j+1]] = g[im[i][j]][im[i][j+1]] + 1
print(normalize(g))

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.27169424 0.2571392  0.72775244]
 [0.         0.         0.         ... 0.27270744 0.25634499 0.68722275]
 [0.         0.         0.         ... 0.22563921 0.2398527  0.79773231]]


In [20]:
def glcm0(d,images):
    g_imgs = []
    for image in images:
        g = np.zeros((256,256))
        for i in range(image.shape[0]):
            for j in range(image.shape[1]-d):
                g[image[i][j]][image[i][j+d]] = g[image[i][j]][image[i][j+d]] + 1
        g_imgs.append(normalize(g))
    g_imgs = np.array(g_imgs)
    return g_imgs

In [21]:
def glcm45(d,images):
    g_imgs = []
    for image in images:
        g = np.zeros((256,256))
        for i in range(d,image.shape[0]):
            for j in range(image.shape[1]-d):
                g[image[i][j]][image[i-d][j+d]] = g[image[i][j]][image[i-d][j+d]] + 1
        g_imgs.append(normalize(g))
    g_imgs = np.array(g_imgs)
    return g_imgs

In [22]:
def glcm90(d,images):
    g_imgs = []
    for image in images:
        g = np.zeros((256,256))
        for i in range(d,image.shape[0]):
            for j in range(image.shape[1]):
                g[image[i][j]][image[i-d][j]] = g[image[i][j]][image[i-d][j]] + 1
        g_imgs.append(normalize(g))
    g_imgs = np.array(g_imgs)
    return g_imgs

In [23]:
def glcm135(d,images):
    g_imgs = []
    for image in images:
        g = np.zeros((256,256))
        for i in range(d,image.shape[0]):
            for j in range(d,image.shape[1]):
                g[image[i][j]][image[i-d][j-d]] = g[image[i][j]][image[i-d][j-d]] + 1
        g_imgs.append(normalize(g))
    g_imgs = np.array(g_imgs)
    return g_imgs

In [24]:
gl1_0 = glcm0(1,X)
print(gl1_0[0])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.27169424 0.2571392  0.72775244]
 [0.         0.         0.         ... 0.27270744 0.25634499 0.68722275]
 [0.         0.         0.         ... 0.22563921 0.2398527  0.79773231]]


In [25]:
gl3_0 = glcm0(3,X)
print(gl3_0[0])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.2075092  0.19952808 0.52675412]
 [0.         0.         0.         ... 0.2414152  0.16381745 0.60353799]
 [0.         0.         0.         ... 0.18313121 0.17997378 0.53360647]]


In [26]:
gl1_45 = glcm45(1,X)
print(gl1_45[0])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.17678192 0.22289894 0.52265958]
 [0.         0.         0.         ... 0.19503435 0.22047361 0.55118403]
 [0.         0.         0.         ... 0.1642795  0.19470163 0.50500737]]


In [27]:
gl2_90 = glcm90(2,X)
print(gl2_90[0])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.17730472 0.10429689 0.40675788]
 [0.         0.         0.         ... 0.20465119 0.19328168 0.45478042]
 [0.         0.         0.         ... 0.17353947 0.15184703 0.43818715]]


In [28]:
gl1_135 = glcm135(1,X)
print(gl1_135[0])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.19308587 0.19308587 0.51242019]
 [0.         0.         0.         ... 0.21831018 0.26029291 0.4450169 ]
 [0.         0.         0.         ... 0.18916768 0.16672405 0.43284129]]


In [29]:
def get_con(image):
    f=0.0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            f = f + (((i-j)**2)*image[i][j])
    return f

In [30]:
def get_corr(image):
    f=0.0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            f = f + (((i-np.mean(image,axis=0)[i])*(j-np.mean(image,axis=1)[j])*image[i][j])/(np.std(image,axis=0)[i]*np.std(image,axis=1)[j]))
    return f

In [31]:
def get_features(g_imgs):
    features=[]
    for img in g_imgs:
        con = get_con(img)
        #corr = get_corr(img)
        en = np.nansum(np.square(img))
        hom = -np.nansum(img*np.log2(img))
        features.append([con,en,hom])
    return np.array(features)

In [32]:
fea0 = get_features(gl1_0)
print(fea0.shape)

<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: divide by zero encountered in log2
  hom = -np.nansum(img*np.log2(img))
<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: invalid value encountered in multiply
  hom = -np.nansum(img*np.log2(img))


(1000, 3)


In [33]:
fea1 = get_features(gl3_0)
print(fea1.shape)

<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: divide by zero encountered in log2
  hom = -np.nansum(img*np.log2(img))
<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: invalid value encountered in multiply
  hom = -np.nansum(img*np.log2(img))


(1000, 3)


In [34]:
fea2 = get_features(gl1_45)
print(fea2.shape)

<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: divide by zero encountered in log2
  hom = -np.nansum(img*np.log2(img))
<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: invalid value encountered in multiply
  hom = -np.nansum(img*np.log2(img))


(1000, 3)


In [35]:
fea3 = get_features(gl2_90)
print(fea3.shape)

<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: divide by zero encountered in log2
  hom = -np.nansum(img*np.log2(img))
<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: invalid value encountered in multiply
  hom = -np.nansum(img*np.log2(img))


(1000, 3)


In [36]:
fea4 = get_features(gl1_135)
print(fea4.shape)

<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: divide by zero encountered in log2
  hom = -np.nansum(img*np.log2(img))
<ipython-input-31-db98b1396ec9>:7: RuntimeWarning: invalid value encountered in multiply
  hom = -np.nansum(img*np.log2(img))


(1000, 3)


In [37]:
fea = np.concatenate((fea0,fea1),axis=1)
fea = np.concatenate((fea,fea2),axis=1)
fea = np.concatenate((fea,fea3),axis=1)
fea = np.concatenate((fea,fea4),axis=1)
fea.shape

(1000, 15)

In [38]:
def train_validate_test_split(X, y, testRatio):
    T_rand = np.random.rand(X.shape[0])
    split = T_rand < np.percentile(T_rand, testRatio * 100)
    X_train = X[~split]
    y_train = y[~split]
    X_test = X[split]
    y_test = y[split]

    return X_train, y_train, X_test, y_test

In [39]:
X_train, y_train, X_test, y_test = train_validate_test_split(fea,Y,0.3)

In [40]:
knn = KNeighborsClassifier(1, metric='euclidean')
knn.fit(X_train, y_train)
prediction = knn.predict(X_test)
print("Accuracy Score =", accuracy_score(y_test, prediction) * 100, "%")

Accuracy Score = 15.666666666666668 %


In [42]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def most_common(lst):
    return max(set(lst), key=lst.count)
def euclidean(point, data):
    # Euclidean distance between points a & data
    return np.sqrt(np.sum((point - data)**2, axis=1))
class KNeighborsClassifier:
    def __init__(self, k=5, dist_metric=euclidean):
        self.k = k
        self.dist_metric = dist_metric
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    def predict(self, X_test):
        neighbors = []
        for x in X_test:
            distances = self.dist_metric(x, self.X_train)
            y_sorted = [y for _, y in sorted(zip(distances, self.y_train))]
            neighbors.append(y_sorted[:self.k])
        return list(map(most_common, neighbors))
    def Accuracy(self, X_test, y_test):
        y_pred = self.predict(X_test)
        acc = sum(y_pred == y_test) / len(y_test)
        return acc

In [45]:

knn = KNeighborsClassifier(k=1)
knn.fit(X_train, y_train)
accuracy = knn.Accuracy(X_test, y_test)

print(accuracy)

0.15666666666666668
